In [1]:
from ImpliedVolatility import ImpliedVolatility                                                                                 
import pandas as pd                                                                                                             
import numpy as np                                                                                                              
import datetime                                                                                                                 
                                                                                                                                
                                                                                                                                
def performance(df, method, r = 0.05, sigma0 = 0.5):                                                                            
    time_start = datetime.datetime.now()                                                                                        
    dfCopy = df.copy()                                                                                                          
                                                                                                                                
    for i in df.index:                                                                                                          
        impVol = ImpliedVolatility(df.ix[i, 'StockPrice'], df.ix[i, 'StrikePrice'], df.ix[i, 'T'], r, sigma0,                   
                                   df.ix[i, 'Last'], df.ix[i, 'Type'])                                                          
                                                                                                                                
        if method == 'bisection':                                                                                               
            dfCopy.ix[i, method] = impVol.bsmBisectionVol()                                                                     
                                                                                                                                
        elif method == 'newton':                                                                                                
            dfCopy.ix[i, method] = impVol.bsmNewtonVol()                                                                        
                                                                                                                                
        elif method == 'mullerBisection':                                                                                       
            dfCopy.ix[i, method] = impVol.bsmMullerBisectionVol()                                                               
                                                                                                                                
        elif method == 'halley':                                                                                                
            dfCopy.ix[i, method] = impVol.bsmHalley()                                                                           
                                                                                                                                
        elif method == 'new_newton':                                                                                            
            impVol.bsmMullerBisectionInitial()                                                                                  
            dfCopy.ix[i, method] = impVol.bsmNewtonVol()                                                                        
                                                                                                                                
        elif method == 'new_halley':                                                                                            
            impVol.bsmMullerBisectionInitial()                                                                                  
            dfCopy.ix[i, method] = impVol.bsmHalley()                                                                           
                                                                                                                                
    dfCopy.loc[:, 'Difference'] = -np.log(np.fabs(dfCopy.IV - dfCopy.ix[:, method]))                        
                                                                                                                                
    accuracy = dfCopy.Difference.mean()                                                                                         
    duration = datetime.datetime.now() - time_start                                                                             
                                                                                                                                
    return accuracy, duration                                                                                                   
                                                                                                                                
                                                                                                                                
data = pd.read_csv('europeanOptions.csv')                                                                                       
data = data.loc[:, ['cDate', 'ExpDate', 'StrikePrice', 'Ticker', 'Type', 'Last', 'IV', 'StockPrice', 'T']]                      
                                                                                                                                
testMethods = ['bisection', 'mullerBisection', 'newton', 'halley', 'new_newton', 'new_halley']                                  
testSigma = [0.1, 0.3, 0.5, 0.7, 1]                                                                                             
result = pd.DataFrame(index=testMethods, columns=['Accuracy', 'Time(hh:mm:ss)'])                                                
resultDic = {}                                                                                                                  
                                                                                                                                
                                                                                                                                
for sigma in testSigma:                                                                                                         
    for method in testMethods:                                                                                                  
        result.loc[method, :] = performance(data, method, sigma0=sigma)                                                         
                                                                                                                                
    resultDic[sigma] = result                                                                                                   
    print('This is the test result for initial point sigma {}:\n{}\n\n'.format(sigma, result))

This is the test result for initial point sigma 0.1:
                  Accuracy  Time(hh:mm:ss)
bisection          3.99595  0:00:21.091301
mullerBisection    3.99246  0:00:21.834244
newton             3.85335  0:15:37.149187
halley           0.0126243  0:00:38.580257
new_newton         4.03708  0:00:39.375585
new_halley         3.95494  0:00:21.692258




This is the test result for initial point sigma 0.3:
                Accuracy  Time(hh:mm:ss)
bisection        3.99595  0:00:20.593348
mullerBisection  3.99246  0:00:21.671891
newton           4.01074  0:01:57.230624
halley           3.68253  0:00:21.559519
new_newton       4.03708  0:00:39.530243
new_halley       3.95494  0:00:21.596327




This is the test result for initial point sigma 0.5:
                Accuracy  Time(hh:mm:ss)
bisection        3.99595  0:00:20.003411
mullerBisection  3.99246  0:00:21.671793
newton           4.03019  0:00:35.345667
halley           3.72324  0:00:14.161432
new_newton       4.03708  0:00:39.362093
new_halley       3.95494  0:00:21.537102




This is the test result for initial point sigma 0.7:
                Accuracy  Time(hh:mm:ss)
bisection        3.99595  0:00:20.069090
mullerBisection  3.99246  0:00:21.640811
newton           4.03589  0:00:26.221396
halley           3.49094  0:00:52.370810
new_newton       4.03708  0:00:39.728422
new_halley       3.95494  0:00:21.696220




This is the test result for initial point sigma 1:
                Accuracy  Time(hh:mm:ss)
bisection        3.99595  0:00:20.178249
mullerBisection  3.99246  0:00:21.844139
newton            4.0434  0:00:33.748404
halley           3.24607  0:00:21.751491
new_newton       4.03708  0:00:39.371573
new_halley       3.95494  0:00:21.432394




In [16]:
for i in resultDic:
    for ind in resultDic[i].index:
        resultDic[i].loc[ind, 'Seconds'] = resultDic[i].loc[ind, 'Time(hh:mm:ss)'].seconds

In [28]:
%matplotlib 
for i in resultDic:
    resultDic[i].plot(kind='hist', subplots=True)

Using matplotlib backend: MacOSX
